In [1]:
!pip install pypdf langchain-openai langchain-core langchain-community langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 21.2 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['DART_API_KEY'] = userdata.get('DART_API_KEY')

In [4]:
# 필요한 라이브러리
import pandas as pd
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os

# 환경변수에서 API 키
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 설정
PINECONE_INDEX_NAME = "samsung-rag"
PINECONE_DIMENSION = 1536
PINECONE_REGION = "us-east-1"
PINECONE_CLOUD = "aws"
EMBEDDING_MODEL = "text-embedding-3-small"
CSV_PATH = "/content/통합_사업보고서 (2022-2024) - 복사본.csv"

# 1. Pinecone 초기화 및 인덱스 생성
pc = Pinecone(api_key=PINECONE_API_KEY)
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION)
    )

# 2. CSV 로드 및 텍스트 변환
df = pd.read_csv(CSV_PATH)
texts = []

for idx, row in df.iterrows():
    row_text = "\n".join([f"{col}: {row[col]}" for col in df.columns if pd.notna(row[col])])
    texts.append(Document(page_content=row_text))

# 3. 텍스트 청크 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(texts)

# 4. 임베딩 + Pinecone 연결
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
index = pc.Index(PINECONE_INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# 5. Pinecone에 업로드 (배치 처리)
BATCH_SIZE = 100
for i in range(0, len(split_docs), BATCH_SIZE):
    batch = split_docs[i:i + BATCH_SIZE]
    vector_store.add_documents(batch)
    print(f"Uploaded {i + len(batch)} / {len(split_docs)} documents")

# 6. 검색 질의 실행
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
results = retriever.invoke("삼성전자 매출총이익은?")

# 7. 결과 출력
for r in results:
    print(r.page_content)
    print("-" * 50)


Uploaded 100 / 86960 documents
Uploaded 200 / 86960 documents
Uploaded 300 / 86960 documents
Uploaded 400 / 86960 documents
Uploaded 500 / 86960 documents
Uploaded 600 / 86960 documents
Uploaded 700 / 86960 documents
Uploaded 800 / 86960 documents
Uploaded 900 / 86960 documents
Uploaded 1000 / 86960 documents
Uploaded 1100 / 86960 documents
Uploaded 1200 / 86960 documents
Uploaded 1300 / 86960 documents
Uploaded 1400 / 86960 documents
Uploaded 1500 / 86960 documents
Uploaded 1600 / 86960 documents
Uploaded 1700 / 86960 documents
Uploaded 1800 / 86960 documents
Uploaded 1900 / 86960 documents
Uploaded 2000 / 86960 documents
Uploaded 2100 / 86960 documents
Uploaded 2200 / 86960 documents
Uploaded 2300 / 86960 documents
Uploaded 2400 / 86960 documents
Uploaded 2500 / 86960 documents
Uploaded 2600 / 86960 documents
Uploaded 2700 / 86960 documents
Uploaded 2800 / 86960 documents
Uploaded 2900 / 86960 documents
Uploaded 3000 / 86960 documents
Uploaded 3100 / 86960 documents
Uploaded 3200 / 8

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA


system_prompt = """
너는 기업의 공시문서(예: 분기보고서, 사업보고서, 감사보고서 등)를 기반으로 정확하고 근거 있는 답변을 제공하는 AI 어시스턴트이자
기업의 현황과 흐름을 알려주는 전문 컨설턴트야.

 지켜야 할 원칙:
- 반드시 제공된 문서 내용만 바탕으로 답변해야 해. 추측하지 마.
- 숫자나 수치는 문서에서 직접 인용하고, 단위를 포함해줘.
- 문서에 근거가 없으면 "해당 정보는 문서에서 찾을 수 없습니다."라고 답변해.
- 가능한 경우, 문서의 출처(ex. 페이지 번호 or 문단 내용)를 함께 포함해.
- 모르는 내용은 절대로 대답하지마


🧾 예시:
"2025년 1분기 삼성전자의 매출총이익은 약 20조 원이며, 이는 2024년 동기 대비 8% 증가한 수치입니다. (출처: 3페이지)"

목표는 **정확성, 투명성, 신뢰도**를 갖춘 답변이야.
"""

# 1. 프롬프트 구성
# 프롬프트 수정: context 변수 포함
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{question}"),
    ("system", "관련 문서:\n{context}")
])
# 2. LLM 구성
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=os.environ["OPENAI_API_KEY"]
)

# 3. QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",  # 문서 결합 방식
    chain_type_kwargs={"prompt": prompt}
)

# 4. 질의
qa_chain.run("현대 포스코의 GM Battery Raw Materials Corporation과 어떤 약정이 있나요? 숫자로 구분지어 대답해주세요")

'1. 현대 포스코는 GM Battery Raw Materials Corporation(이하 GM)과 GP Shareholders Agreement를 체결하였습니다.\n2. 본 계약 서명일로부터 2년째 되는 날 이전에 산업 기술의 유출 방지 및 보호에 관한 법률에 따라 무조건적인 국가 핵심기술 수출 승인을 취득하지 않은 경우, GM은 연결실체에 지분 전부(GM이 보유한 ULTIUM CAM GP INC. 및 ULTIUM CAM LIMITED PARTNERSHIP)를 매각할 수 있습니다.\n3. GM은 본 계약 서명일로부터 27개월째 날까지 이러한 풋옵션을 행사할 수 있으며, 풋옵션 행사 시 연결실체는 (i) GM의 지분 공정시장가액 또는 (ii) GM이 매각 마감 전에 수행한 자본 출자 총액의 125% 중 더 큰 금액으로 GM의 지분을 즉시 구매해야 합니다. (출처: 문서 내용)'

In [11]:
import pandas as pd

df= pd.read_csv('/content/통합_사업보고서 (2022-2024) - 복사본.csv')
df

,년도,회사명,텍스트 미리보기
0,2021,삼성전자,감사보고서\n3.8\n삼성전자(주)\n00260082\n100000000000\n2...
1,2022,삼성전자,분기보고서\n4.8\n삼성전자주식회사\n130111-0006246\n분 기 보 고 ...
2,2023,삼성전자,분기보고서\n5.0\n삼성전자주식회사\n130111-0006246\n분 기 보 고 ...
3,2024,삼성전자,분기보고서\n5.6\n삼성전자주식회사\n1\nY\n130111-0006246\n분 ...
4,2021,SK하이닉스,연결감사보고서\n3.4\n에스케이하이닉스주식회사\n00260295\n10000000...
...,...,...,...
216,2024,포스코퓨처엠,분기보고서\n5.6\n포스코퓨처엠\n1\nY\n174611-0001947\n분 기 ...
217,2021,HD현대,감사보고서\n3.8\n현대중공업지주 주식회사\n00260116\n1000000000...
218,2022,HD현대,분기보고서\n4.8\n에이치디현대주식회사\n170111-0638661\n분 기 보 ...
219,2023,HD현대,분기보고서\n5.0\n에이치디현대주식회사\n170111-0638661\n분 기 보 ...


In [14]:
df['텍스트 미리보기'].apply(len)

,텍스트 미리보기
0,94851
1,324814
2,328024
3,362582
4,117644
...,...
216,180995
217,81244
218,502347
219,480254


In [22]:
df['텍스트 미리보기'][216]

'분기보고서\n5.6\n포스코퓨처엠\n1\nY\n174611-0001947\n분 기 보 고 서\n(제 54 기 1분기)\n사업연도\n2024년 01월 01일\n부터\n2024년 03월 31일\n까지\n금융위원회\n한국거래소 귀중\n2024년 5월 14일\n제출대상법인 유형 :\n주권상장법인\n면제사유발생 :\n해당사항 없음\n회 사 명 :\n(주)포스코퓨처엠\n대 표 이 사 :\n유 병 옥\n본 점 소 재 지 :\n포항시 남구 신항로 110\n(전 화) 054-290-0114\n(홈페이지) http://www.poscofuturem.com/\n작 성 책 임 자 :\n(직 책) 경영기획실장 (성 명) 정 연 수\n(전 화) 054-290-0140\n목 차\n【 대표이사 등의 확인 】\n대표이사등의 확인서_240514.jpg\n대표이사등의 확인서_240514\nI. 회사의 개요\n1. 회사의 개요\n회사의 개요는 기업공시서식 작성기준에 따라 분기보고서에 기재하지 않습니다.\n(반기, 사업보고서에 기재 예정)\n2. 회사의 연혁\n회사의 연혁은 기업공시서식 작성기준에 따라 분기보고서에 기재하지 않습니다.\n(반기, 사업보고서에 기재 예정)\n3. 자본금 변동사항\n자본금 변동사항은 기업공시서식 작성기준에 따라 분기보고서에 기재하지 않습니다.\n(반기, 사업보고서에 기재 예정)\n4. 주식의 총수 등\n주식의 총수 등은 기업공시서식 작성기준에 따라 분기보고서에 기재하지 않습니다.\n(반기, 사업보고서에 기재 예정)\n5. 정관에 관한 사항\n정관에 관한 사항은 기업공시서식 작성기준에 따라 분기보고서에 기재하지 않습니다.\n(반기, 사업보고서에 기재 예정)\nII. 사업의 내용\n1. 사업의 개요\n포스코퓨처엠은 국내 유일의 흑연계 음극재 회사이자 155천 톤/년(2024년 3월, 해외 포함)의 양극재 생산능력을 보유한 국내 유일 음/양극재 소재를 동시 생산 및 납품하는 이차전지 소재사입니다. 포스코퓨처엠은 1963년 1월 염기성내화물을 생산,판매 목적으로 설립된 삼화